<a href="https://colab.research.google.com/github/wakamatsuikuma/textMEMO_causal_analysis_by_python/blob/main/%E7%AC%AC4%E7%AB%A0_%E5%9B%A0%E6%9E%9C%E6%8E%A8%E8%AB%96%E3%82%92%E5%AE%9F%E8%A3%85%E3%81%97%E3%82%88%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ
- 調整化公式：doオペレータで表した数式を、doオペレータ無しで表現する方法→これだけでは考慮すべき変数（共変量）がわからない
- d分離する：適切に因果ダイアグラムの変数を無視し、さらに介入を実施して因果推論するために因果ダイアグラムを変更する操作(=共変量の推定)
- 傾向スコアによる逆確率重み付法(IPTW)：原因変数を共変量でモデリングし、共変量という条件での原因変数の確率(ある属性における処置zである確率、傾向スコア)を算出。ATEを調整化公式で変形した数式に傾向スコアを代入し処置の推定効果を算出する。
- DR法(Doubly Robust)：IPTWでは各サンプルにおいて反実仮想側の結果を無視しているので、結果変数を回帰モデルなどで推定して(=反実仮想における結果の推定)、ATEの算出に反実仮想での逆確率重み付の項を追加して、推定効果を算出する。

# 4-1 回帰分析による因果推論の実装

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import numpy as np

# 乱数のシードを固定
random.seed(1234)
np.random.seed(1234)

# Scipy 平均0、分散1に正規化(標準化)関数
import scipy.stats

# シグモイド関数をimport
from scipy.special import expit

# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt
%matplotlib inline

# その他
import pandas as pd

## 説明変数の生成

In [ ]:
# データ数
num_data = 200

# 原因変数以外の説明変数の生成
# 年齢
x_1 = randint(15, 76, num_data) 

# 性別
x_2 = randint(0, 2, num_data)


In [ ]:
# 原因変数の生成
# ノイズ(バイアス項)の生成
e_z = randn(num_data)

# シグモイド関数に入れる部分。テレビCMを見る可能性を確率変数(?、的なもの)として生成。(大きいほど見る確率は高くなる)
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# テレビCMを見る確率を計算 = シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])
for i in range(num_data):
  Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0] # 0または1になる確率に対して、ランダムに選択している？
  Z = np.append(Z, Z_i)

## 目的変数の生成

In [ ]:
# ノイズ(バイアス項)の生成
e_y = randn(num_data)

# 各係数(=効果)も指定して生成。年齢が高いほど減少し、男性の方が増加し、テレビCMを見ていると増加する。
Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

## テーブルを作成

In [ ]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

df.head()  # 先頭を表示

,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [ ]:
# 平均値を比べる

print(df[df["CMを見た"] == 1.0].mean())
print("--------")
print(df[df["CMを見た"] == 0.0].mean())

年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
--------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


## 分析を実施

In [ ]:
# scikit-learnから線形回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.linear_model import LinearRegression

# 説明変数
X = df[["年齢", "性別", "CMを見た"]]

# 被説明変数（目的変数）
y = df["購入量"]

# 回帰の実施
reg = LinearRegression().fit(X, y)

# 回帰した結果の係数を出力
print("係数：", reg.coef_)


係数： [-0.95817951 32.70149412 10.41327647]


In [ ]:
# 以下でもいける
model = LinearRegression()
model.fit(X, y)
model.coef_

array([-0.95817951, 32.70149412, 10.41327647])

# 4-2 傾向スコアを用いた逆確率重み付けほう(IPTW)の実装

In [ ]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats

# シグモイド関数をimport
from scipy.special import expit

# その他
import pandas as pd

## 説明変数の生成

In [ ]:
# データ数
num_data = 200

# 原因変数以外の説明変数の生成
# 年齢
x_1 = randint(15, 76, num_data) 

# 性別
x_2 = randint(0, 2, num_data)

In [ ]:
# 原因変数の生成
# ノイズ(バイアス項)の生成
e_z = randn(num_data)

# シグモイド関数に入れる部分。テレビCMを見る可能性を確率変数(?)として生成。(大きいほど見る確率は高くなる)
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# テレビCMを見る確率を計算 = シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])
for i in range(num_data):
  Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0] # 0または1になる確率に対して、ランダムに選択している？
  Z = np.append(Z, Z_i)

## 目的変数の生成

In [ ]:
# ノイズ(バイアス項)の生成
e_y = randn(num_data)

# 各係数(=効果)も指定して生成。年齢が高いほど減少し、男性の方が増加し、テレビCMを見ていると増加する。
Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

## テーブルの生成

In [ ]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

df.head()  # 先頭を表示

,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [ ]:
# 平均値を比べる

print(df[df["CMを見た"] == 1.0].mean())
print("--------")
print(df[df["CMを見た"] == 0.0].mean())

年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
--------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


## 分析を実施

In [ ]:
# モデリング

# scikit-learnからロジスティク回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression

# 傾向スコアを求める上では、原因変数を目的変数として分析を実施
# 傾向スコア = 他の説明変数の値をとる上での、介入がある値(ここでは0 or 1)をとる確率　を求めているから！

# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X,Z)

# 回帰した結果の係数を出力
print("係数beta：", reg.coef_)
print("係数alpha：", reg.intercept_)

係数beta： [[ 0.10562765 -1.38263933]]
係数alpha： [-3.37146523]


In [ ]:
# 各人の傾向スコアを計算

Z_pred = reg.predict_proba(X)
print(Z_pred[0:5])  # 5人ほどの結果を見てみる
print("----")
print(Z[0:5])  # 5人ほどの正解

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]
----
0    1.0
1    0.0
2    1.0
3    1.0
4    0.0
Name: CMを見た, dtype: float64


In [ ]:
# 平均処置効果 ATEを計算

ATE_i = Y / Z_pred[:, 1] * Z - Y / Z_pred[:, 0] * (1-Z)
ATE = 1 / len(Y) * ATE_i.sum()
print("推定したATE", ATE)

推定したATE 8.847476810855458


- num_data = 1000にすると10.03とかなり正確な値となった
- モデル係数はそこまでだった

In [ ]:
ATE_i

0       25.484247
1     -102.623783
2       92.942211
3       51.347004
4     -115.055932
          ...    
195    -60.182748
196   -125.865433
197    -75.288617
198     57.713402
199    117.800952
Name: CMを見た, Length: 200, dtype: float64

# 4-3 Doubly Robust法(DR法)による因果推論の実装

In [ ]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats

# シグモイド関数をimport
from scipy.special import expit

# その他
import pandas as pd

## 説明変数の生成

In [ ]:
# データ数
num_data = 200

# 原因変数以外の説明変数の生成
# 年齢
x_1 = randint(15, 76, num_data) 

# 性別
x_2 = randint(0, 2, num_data)

In [ ]:
# 原因変数の生成
# ノイズ(バイアス項)の生成
e_z = randn(num_data)

# シグモイド関数に入れる部分。テレビCMを見る可能性を確率変数(?)として生成。(大きいほど見る確率は高くなる)
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# テレビCMを見る確率を計算 = シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])
for i in range(num_data):
  Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0] # 0または1になる確率に対して、ランダムに選択している？
  Z = np.append(Z, Z_i)

## 目的変数の生成

In [ ]:
# ノイズ(バイアス項)の生成
e_y = randn(num_data)

# 各係数(=効果)も指定して生成。年齢が高いほど減少し、男性の方が増加し、テレビCMを見ていると増加する。
Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

## テーブルの生成

In [ ]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

df.head()  # 先頭を表示

,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [ ]:
# 平均値を比べる

print(df[df["CMを見た"] == 1.0].mean())
print("--------")
print(df[df["CMを見た"] == 0.0].mean())

年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
--------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


## 分析の実施

In [ ]:
# scikit-learnから線形回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.linear_model import LinearRegression

# 説明変数
X = df[["年齢", "性別", "CMを見た"]]

# 被説明変数（目的変数）
y = df["購入量"]

# まずは回帰で、反実仮想の推定値にあたる購入量を予測する
# 回帰のモデリング
reg2 = LinearRegression().fit(X, y)

# 介入Z=0(テレビCMを見ていない)の場合の購入量を予測
X_0 = X.copy()
X_0["CMを見た"] = 0
Y_0 = reg2.predict(X_0)

# 介入Z=1(テレビCMを見た)の場合の購入量を予測
X_1 = X.copy()
X_1["CMを見た"] = 1
Y_1 = reg2.predict(X_1)

In [ ]:
# scikit-learnからロジスティク回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression

# 傾向スコアを求める上では、原因変数を目的変数として分析を実施
# 傾向スコア = 他の説明変数の値をとる上での、介入がある値(ここでは0 or 1)をとる確率を求めているから！

# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X, Z)

# 傾向スコアを求める
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])  # 5人ほどの結果を見てみる

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]


In [ ]:
# do(Z=1)、do(Z=0)の項ともに、反実仮想の介入サンプルの推定量を活用するために足している感じ
# 観察データにあるサンプルも一括で推定して、加えているのはよくわからん
# → ATE_1_i, ATE_0_iのそれぞれを前半を観察結果、後半を反実仮想の出力と考えると、
#    反実仮想側の観察結果で取得されている介入有りのサンプルは、1-Zで0にされている

ATE_1_i = Y/Z_pre[:, 1]*Z + (1-Z/Z_pre[:, 1])*Y_1
ATE_0_i = Y/Z_pre[:, 0]*(1-Z) + (1-(1-Z)/Z_pre[:, 0])*Y_0
ATE = 1/len(Y)*(ATE_1_i-ATE_0_i).sum()
print("推定したATE", ATE)

推定したATE 9.75277505424846


- num_data=1000にしたら、実際の効果からより離れた